# Fine tune with full scale dataset

## Import and utilities

In [ ]:
import torch
import os
import sys

from transformers import Trainer, TrainingArguments

/jet/home/iwiryadi/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/jet/home/iwiryadi/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/jet/home/iwiryadi/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
GIT_URL = "https://github.com/strivn/idl-project"

In [ ]:
current_dir = os.path.abspath(os.getcwd())

# Check if 'idl-project' is in the path
if 'idl-project' not in current_dir:
    raise Exception("Current directory '{current_dir}' is not within 'idl-project'")

print(f"✓ Working in '{current_dir}'")
print(f"✓ Directory contains 'idl-project'")

✓ Working in '/ocean/projects/cis250068p/iwiryadi/idl-project'
✓ Directory contains 'idl-project'


In [ ]:
!git pull f"{GIT_URL}"

git: 'remote-fhttps' is not a git command. See 'git --help'.

The most similar command is
	remote-https


In [ ]:
# Check CUDA
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla V100-SXM2-32GB


In [ ]:
# Append paths for the src folder
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'idl-project')))

In [ ]:
# Additional imports 

from src.model import load_fo_model
from src.data import load_flan_dataset

Using device: cuda


In [ ]:
OUTPUT_DIR = "output/"

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

## Load model and dataset

In [ ]:
model, tokenizer = load_fo_model()

/jet/home/iwiryadi/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
batch_size = 128
dataset = load_flan_dataset().batch(batch_size=batch_size)

Resolving data files:   0%|          | 0/2167 [00:00<?, ?it/s]

Dataset loaded successfully


In [ ]:
print(tokenizer.pad_token)
print(tokenizer.pad_token_id)
print(tokenizer.eos_token)

None
None
<|endoftext|>


In [ ]:
# Configure tokenizer properly 

# - https://github.com/EleutherAI/pythia/issues/156 
#   mentioned it's okay to set it to eos

# - https://huggingface.co/EleutherAI/pythia-14m/discussions/4, 
#   we can see the tokenizer pad token from tokenizer.added_tokens_decoder 
tokenizer.added_tokens_decoder

{0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 1: AddedToken("<|padding|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 50254: AddedToken("                        ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
 50255: AddedToken("                       ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
 50256: AddedToken("                      ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
 50257: AddedToken("                     ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
 50258: AddedToken("                    ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
 50259: AddedToken("                   ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
 50260: AddedToken("           

In [ ]:
# without custom pad token set to zero the convergence behaves weirdly
tokenizer.pad_token    = "<|padding|>"
tokenizer.pad_token_id = 1     

In [ ]:
tokenizer.eos_token

'<|endoftext|>'

In [ ]:
def preprocess_forward(example):
    # combine input, output, and eos token - (I:... Q:... A:... <eos>)
    combined_text = f"{example['inputs']}\n{example['targets']}{tokenizer.eos_token}"
    
    tokenized = tokenizer(
        combined_text,
        truncation=True,
        max_length=768,
        padding="max_length",
        return_tensors="pt"
    )
    
    input_ids = tokenized["input_ids"][0]
    labels = input_ids.clone()
    
    # shift left 
    labels[:-1] = input_ids[1:]
    
    # mask the pad and eos token
    labels[input_ids == tokenizer.pad_token_id] = -100
    labels[input_ids == tokenizer.eos_token_id] = -100
    #TODO: Do we need to mask the prompt tokens?
    
    return {"input_ids": input_ids, "labels": labels}

## Data preprocessing

In [ ]:
def preprocess_forward(examples):
    # Combine inputs, outputs, and eos token for each example in the batch
    combined_texts = []
    
    for inp, tgt in zip(examples['inputs'], examples['targets']):
        # Handle potential None values or empty strings
        inp_text = inp if inp is not None else ""
        tgt_text = tgt if tgt is not None else ""
        combined_texts.append(f"{inp_text}\n{tgt_text}{tokenizer.eos_token}")
    
    # Tokenize the entire batch at once
    tokenized = tokenizer(
        combined_texts,
        truncation=True,
        max_length=768,
        padding="max_length",
        return_tensors="pt"
    )
    
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]
    labels = input_ids.clone()
    
    # Shift labels to the left (for each sequence in the batch)
    labels[:, :-1] = input_ids[:, 1:]
    # Add padding token as the last prediction target
    labels[:, -1] = tokenizer.pad_token_id
    
    # Mask pad and eos tokens in labels
    labels[input_ids == tokenizer.pad_token_id] = -100
    labels[input_ids == tokenizer.eos_token_id] = -100
    
    # You might want to mask the prompt tokens in labels too
    # This is the TODO item from your original code
    # One common approach is to find where the targets start and mask everything before
    
    return {
        "input_ids": input_ids, 
        "attention_mask": attention_mask,
        "labels": labels
    }

## Check

In [ ]:
examples = next(iter(dataset))
combined_texts = []
for inp, tgt in zip(examples['inputs'], examples['targets']):
    # Handle potential None values or empty strings
    inp_text = inp if inp is not None else ""
    tgt_text = tgt if tgt is not None else ""
    combined_texts.append(f"{inp_text}\n{tgt_text}{tokenizer.eos_token}")

In [ ]:
print(combined_texts.__len__())
print(combined_texts[9])

256
The women can't be competing in a martial arts match and fighting over the same man at the same time.
The question and answer are below.
Premise: "Two women are competing in a martial arts match where one woman is pinning the other by sitting on her head."
Hypothesis: "2 women are fighting over the same man in a duel to the death."
Do we know that the hypothesis entailed by the premise?
no

The children are the two boys and a girl who are wearing bright ethnic garb for clothes.. So what could be the question?
Question followed by answer: Given the sentence "A girl and two boys dressed in bright ethnic garb are walking together." can we conclude that "Children in bright clothes are walking."?
Options:
- yes
- it is not possible to tell
- no
yes

Someone is using paint because he is painting a model for a photo shoot.
The question and answer are below.
Can we conclude from "A man is painting a message on a model's arm during a photo shoot and a glass of wine is sitting on the floor."

## Training Setup

In [ ]:
# Configuration
total_examples = 378_000_000  # Total examples in FLAN dataset
train_ratio = 0.9
val_ratio = 0.05
test_ratio = 0.05

In [ ]:
# Shuffle and split before preprocessing
shuffled = dataset.shuffle(seed=42, buffer_size=100_000)

# Calculate split sizes
train_size = int(total_examples * train_ratio)
val_size = int(total_examples * val_ratio)
test_size = int(total_examples * test_ratio)

# Create splits
train_raw = shuffled.take(train_size)
remaining = shuffled.skip(train_size)
val_raw = remaining.take(val_size)
test_raw = remaining.skip(val_size).take(test_size)

# Preprocess each split
tokenized_train = train_raw.map(preprocess_forward, batched=True)
tokenized_val = val_raw.map(preprocess_forward, batched=True)
tokenized_test = test_raw.map(preprocess_forward, batched=True)

In [ ]:
import datasets
datasets.enable_progress_bar()

In [ ]:
# Training configuration
intended_epochs = 5
gradient_accum_steps = 4
max_steps =int(total_examples * train_ratio) // (batch_size * gradient_accum_steps) * intended_epochs
print(max_steps)

1661130


In [ ]:
training_args = TrainingArguments(
    output_dir=f"{OUTPUT_DIR}/pythia-finetuned",
    evaluation_strategy="steps",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    max_steps=max_steps,  # Required for streaming datasets
    
    
    eval_steps=100,
    save_steps=100,
    logging_steps=gradient_accum_steps,
    gradient_accumulation_steps=gradient_accum_steps,
    
    save_total_limit=2,
    
    weight_decay=0.01,
    fp16=True,
    
    logging_dir="./logs",
    # report_to="none",
    report_to="wandb",
    push_to_hub=False,
    disable_tqdm=False,
    
    dataloader_num_workers = 4,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

# Start training
trainer.train()

# Save final model
trainer.save_model(f"{OUTPUT_DIR}/pythia-finetuned-final")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/pythia-finetuned-final")

RuntimeError: DataLoader worker (pid(s) 9151) exited unexpectedly

# Reversed 

In [ ]:
dataset = load_dataset("chiayewken/flan-v2", split="train", streaming=True)

model_name = "afterless/reverse-pythia-160m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = "<pad>"
tokenizer.pad_token_id = 0

def preprocess_reverse(example):
    combined_text = f"{example['source']}\n{example['target']}"

    tokenized_output = tokenizer(
        combined_text,
        truncation=True,
        padding="max_length",
        max_length=768,
        return_tensors="pt",
    )

    input_ids = tokenized_output["input_ids"].squeeze(0)  # Remove batch dim
    reversed_input_ids = input_ids.flip(dims=[0])  # Reverse sequence

    labels = reversed_input_ids.clone()
    labels = torch.roll(labels, shifts=1, dims=0)  # Shift right by 1
    labels[0] = -100  # Ignore loss for first token

    return {
        "input_ids": reversed_input_ids,
        "labels": labels,
    }

for i, example in enumerate(dataset):
    tokenized_example = preprocess_reverse(example)
    print(f"Example {i + 1}:")
    print(f"Input IDs: {tokenized_example['input_ids']}")
    print(f"Decoded Tokens: {tokenizer.decode(tokenized_example['input_ids'], skip_special_tokens=True)}")
    print("-" * 50)

    if i == 2:  # Show only 3 examples
        break
